# Should we start with PET or Biomarkers?
A Monte Carlo Simulation

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Dict, Tuple
from scipy.stats import wilcoxon
import ipywidgets as widgets
from ipywidgets import VBox, HBox, Layout


In [2]:
@dataclass
class Config:
    n_patients_per_run: int = 1000
    n_runs: int = 1000
    prevalence_cancer: float = 0.20
    pet_sens: float = 0.89
    pet_spec: float = 0.75
    # We'll treat "biomarker sens/spec" sliders as CDT sens/spec
    cdt_sens: float = 0.41
    cdt_spec: float = 0.93
    xl2_lb_rate_benign: float = 0.44
    xl2_lb_rate_cancer: float = 0.03

def simulate_one_run(cfg: Config, rng: np.random.Generator) -> Dict[str, int]:
    N = cfg.n_patients_per_run
    cancers = rng.random(N) < cfg.prevalence_cancer
    benigns = ~cancers

    # PET-first
    pet_draw = rng.random(N)
    pet_positive = np.where(cancers, pet_draw < cfg.pet_sens, pet_draw > cfg.pet_spec)
    pet_negative = ~pet_positive
    unnec_bx_petfirst = int(np.sum(pet_positive & benigns))
    cancers_surv_petfirst = int(np.sum(pet_negative & cancers))
    total_pets_petfirst = N
    total_biomarkers_petfirst = 0

    # Biomarker-first: CDT → XL2 → PET if needed
    cdt_draw = rng.random(N)
    cdt_positive = np.where(cancers, cdt_draw < cfg.cdt_sens, cdt_draw > cfg.cdt_spec)
    cdt_negative = ~cdt_positive

    xl2_draw = rng.random(N)
    xl2_lb = np.zeros(N, dtype=bool)
    xl2_lb[cdt_negative & cancers] = xl2_draw[cdt_negative & cancers] < cfg.xl2_lb_rate_cancer
    xl2_lb[cdt_negative & benigns] = xl2_draw[cdt_negative & benigns] < cfg.xl2_lb_rate_benign
    indeterminate_after_xl2 = cdt_negative & (~xl2_lb)

    pet_draw2 = rng.random(N)
    pet_pos_biofirst = np.zeros(N, dtype=bool)
    pet_pos_biofirst[indeterminate_after_xl2 & cancers] = pet_draw2[indeterminate_after_xl2 & cancers] < cfg.pet_sens
    pet_pos_biofirst[indeterminate_after_xl2 & benigns] = pet_draw2[indeterminate_after_xl2 & benigns] > cfg.pet_spec
    pet_neg_biofirst = indeterminate_after_xl2 & (~pet_pos_biofirst)

    biopsy_biofirst = cdt_positive | pet_pos_biofirst
    unnec_bx_biofirst = int(np.sum(biopsy_biofirst & benigns))
    cancers_surv_biofirst = int(np.sum((xl2_lb & cancers) | (pet_neg_biofirst & cancers)))

    decision_pets = int(np.sum(indeterminate_after_xl2))
    staging_pets = int(np.sum(cdt_positive & cancers))
    total_pets_biofirst = decision_pets + staging_pets
    total_biomarkers_biofirst = N + int(np.sum(cdt_negative))

    return {
        "unnec_bx_petfirst": unnec_bx_petfirst,
        "cancers_surv_petfirst": cancers_surv_petfirst,
        "total_pets_petfirst": total_pets_petfirst,
        "total_biomarkers_petfirst": total_biomarkers_petfirst,
        "unnec_bx_biofirst": unnec_bx_biofirst,
        "cancers_surv_biofirst": cancers_surv_biofirst,
        "total_pets_biofirst": total_pets_biofirst,
        "total_biomarkers_biofirst": total_biomarkers_biofirst,
    }

def simulate_many(cfg: Config, seed: int = 42) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    rows = [simulate_one_run(cfg, rng) for _ in range(cfg.n_runs)]
    return pd.DataFrame(rows)

def ci95(series: pd.Series) -> Tuple[float, float, float]:
    return float(series.quantile(0.5)), float(series.quantile(0.025)), float(series.quantile(0.975))

def add_malignancy_rates(df: pd.DataFrame, n_patients: int, prevalence: float) -> pd.DataFrame:
    cancers_total = n_patients * prevalence

    malignant_bx_pet = cancers_total - df["cancers_surv_petfirst"]
    total_bx_pet = df["unnec_bx_petfirst"] + malignant_bx_pet
    df["malig_rate_petfirst"] = np.where(total_bx_pet > 0, 100.0 * malignant_bx_pet / total_bx_pet, np.nan)

    malignant_bx_bio = cancers_total - df["cancers_surv_biofirst"]
    total_bx_bio = df["unnec_bx_biofirst"] + malignant_bx_bio
    df["malig_rate_biofirst"] = np.where(total_bx_bio > 0, 100.0 * malignant_bx_bio / total_bx_bio, np.nan)

    return df

def med_ci(df: pd.DataFrame, col: str):
    med, lo, hi = ci95(df[col])
    return med, med - lo, hi - med  # value, lower_err, upper_err

def format_p(p: float) -> str:
    if p < 0.001:
        return "p < 0.001"
    elif p < 0.01:
        return f"p = {p:.3f}".replace("0.", ".")
    else:
        return f"p = {p:.2f}".replace("0.", ".")


In [25]:
# Sliders
prevalence_slider = widgets.FloatSlider(
    value=0.20, min=0.05, max=0.60, step=0.01,
    description="Prevalence", readout_format=".2f", continuous_update=False
)

bio_sens_slider = widgets.FloatSlider(
    value=0.41, min=0.10, max=0.90, step=0.01,
    description="Bio Sens", readout_format=".2f", continuous_update=False
)

bio_spec_slider = widgets.FloatSlider(
    value=0.93, min=0.50, max=0.99, step=0.01,
    description="Bio Spec", readout_format=".2f", continuous_update=False
)

pet_sens_slider = widgets.FloatSlider(
    value=0.89, min=0.50, max=0.99, step=0.01,
    description="PET Sens", readout_format=".2f", continuous_update=False
)

pet_spec_slider = widgets.FloatSlider(
    value=0.75, min=0.30, max=0.99, step=0.01,
    description="PET Spec", readout_format=".2f", continuous_update=False
)

# Presets for biomarker and PET (as ToggleButtons)
biomarker_preset = widgets.ToggleButtons(
    options=[
        ("Custom biomarker", "custom"),
        ("Published biomarker", "published"),
    ],
    value="published",
    description="Biomarker preset"#,
    #layout=Layout(width="300px")
)

pet_preset = widgets.ToggleButtons(
    options=[
        ("Custom PET", "custom"),
        ("Non-endemic fungal", "non_endemic"),
        ("Endemic fungal (MA)", "endemic_ma"),
        ("Endemic fungal (worst)", "endemic_worst"),
        ("Mixed", "mixed"),
    ],
    value="non_endemic",
    description="PET preset"
    #layout=Layout(width="300px")
)



In [26]:
# Preset values (from your published/sim assumptions)
BIOMARKER_PRESETS = {
    "published": (0.41, 0.93),
}

PET_PRESETS = {
    "non_endemic": (0.89, 0.75),
    "endemic_ma": (0.89, 0.61),
    "endemic_worst": (0.89, 0.40),
    "mixed": (0.89, 0.68),  # example "middle" specificity; tweak if you like
}

# Flags so that when *presets* update sliders, the slider callbacks
# don't flip the preset back to "custom".
_updating_biomarker_from_preset = False
_updating_pet_from_preset = False

def on_biomarker_preset_change(change):
    global _updating_biomarker_from_preset
    if change["name"] != "value":
        return
    key = change["new"]
    if key == "custom":
        return
    sens, spec = BIOMARKER_PRESETS[key]
    _updating_biomarker_from_preset = True
    try:
        bio_sens_slider.value = sens
        bio_spec_slider.value = spec
    finally:
        _updating_biomarker_from_preset = False

def on_pet_preset_change(change):
    global _updating_pet_from_preset
    if change["name"] != "value":
        return
    key = change["new"]
    if key == "custom":
        return
    sens, spec = PET_PRESETS[key]
    _updating_pet_from_preset = True
    try:
        pet_sens_slider.value = sens
        pet_spec_slider.value = spec
    finally:
        _updating_pet_from_preset = False

biomarker_preset.observe(on_biomarker_preset_change, names="value")
pet_preset.observe(on_pet_preset_change, names="value")

# --- New: sliders force preset → "custom" ---

def on_biomarker_slider_change(change):
    if change["name"] != "value":
        return
    # Ignore changes that came from applying a preset
    if _updating_biomarker_from_preset:
        return
    biomarker_preset.value = "custom"

bio_sens_slider.observe(on_biomarker_slider_change, names="value")
bio_spec_slider.observe(on_biomarker_slider_change, names="value")

def on_pet_slider_change(change):
    if change["name"] != "value":
        return
    # Ignore changes that came from applying a preset
    if _updating_pet_from_preset:
        return
    pet_preset.value = "custom"

pet_sens_slider.observe(on_pet_slider_change, names="value")
pet_spec_slider.observe(on_pet_slider_change, names="value")

In [29]:
def run_simulation(prevalence, bio_sens, bio_spec, pet_sens, pet_spec):
    
    cfg = Config(
        prevalence_cancer=prevalence,
        pet_sens=pet_sens,
        pet_spec=pet_spec,
        cdt_sens=bio_sens,
        cdt_spec=bio_spec,
    )
    
    df_base = simulate_many(cfg, seed=1)
    df_base = add_malignancy_rates(df_base, cfg.n_patients_per_run, cfg.prevalence_cancer)
    
    # --- Summaries (medians + 95% CI error components) ---
    benign_pet = med_ci(df_base, "unnec_bx_petfirst")
    benign_bio = med_ci(df_base, "unnec_bx_biofirst")
    
    miss_pet = med_ci(df_base, "cancers_surv_petfirst")
    miss_bio = med_ci(df_base, "cancers_surv_biofirst")
    
    mr_pet = med_ci(df_base, "malig_rate_petfirst")
    mr_bio = med_ci(df_base, "malig_rate_biofirst")
    
    total_pet_pet = med_ci(df_base, "total_pets_petfirst")
    total_pet_bio = med_ci(df_base, "total_pets_biofirst")
    
    # --- Paired p-values ---
    p_benign = wilcoxon(df_base["unnec_bx_petfirst"] - df_base["unnec_bx_biofirst"]).pvalue
    p_miss   = wilcoxon(df_base["cancers_surv_petfirst"] - df_base["cancers_surv_biofirst"]).pvalue
    p_mr     = wilcoxon(df_base["malig_rate_petfirst"] - df_base["malig_rate_biofirst"]).pvalue
    p_pets   = wilcoxon(df_base["total_pets_petfirst"] - df_base["total_pets_biofirst"]).pvalue
    
    
    
    # --- 4-panel plot: malignancy, benign biopsies, PETs, cancers in surveillance ---
    fig, axes = plt.subplots(1, 4, figsize=(12, 4))  # no constrained_layout
    #fig, axes = plt.subplots(2, 2, figsize=(10, 6))
    
    labels = ["PET-first", "Biomarker-first"]
    colors = ["tab:blue", "tab:orange"]
    
    # Panel 0: malignancy among biopsies (%)
    vals_mr = [mr_pet[0], mr_bio[0]]
    err_mr  = [[mr_pet[1], mr_bio[1]],
               [mr_pet[2], mr_bio[2]]]
    axes[0].bar([0, 1], vals_mr, yerr=err_mr, capsize=5, color=colors)
    axes[0].set_xticks([0, 1])
    axes[0].set_xticklabels(labels, rotation=0)
    axes[0].set_ylabel("Malignancy among biopsies (%)")
    axes[0].set_title("Biopsy yield")
    
    # Panel 1: benign biopsies per 1000
    vals_benign = [benign_pet[0], benign_bio[0]]
    err_benign  = [[benign_pet[1], benign_bio[1]],
                   [benign_pet[2], benign_bio[2]]]
    axes[1].bar([0, 1], vals_benign, yerr=err_benign, capsize=5, color=colors)
    axes[1].set_xticks([0, 1])
    axes[1].set_xticklabels(labels, rotation=0)
    axes[1].set_ylabel("Benign biopsies per 1000")
    axes[1].set_title("Benign biopsies")
    
    # Panel 2: total PET scans
    vals_pets = [total_pet_pet[0], total_pet_bio[0]]
    err_pets  = [[total_pet_pet[1], total_pet_bio[1]],
                 [total_pet_pet[2], total_pet_bio[2]]]
    axes[2].bar([0, 1], vals_pets, yerr=err_pets, capsize=5, color=colors)
    axes[2].set_xticks([0, 1])
    axes[2].set_xticklabels(labels, rotation=0)
    axes[2].set_ylabel("PET scans per 1000")
    axes[2].set_title("PET utilization")
    
    # Panel 3: cancers in surveillance
    vals_miss = [miss_pet[0], miss_bio[0]]
    err_miss  = [[miss_pet[1], miss_bio[1]],
                 [miss_pet[2], miss_bio[2]]]
    axes[3].bar([0, 1], vals_miss, yerr=err_miss, capsize=5, color=colors)
    axes[3].set_xticks([0, 1])
    axes[3].set_xticklabels(labels, rotation=0)
    axes[3].set_ylabel("Cancers in surveillance")
    axes[3].set_title("Missed cancers")
    
    # ---- p-values above each panel (axis coords so they never clip) ----
    p_texts = [format_p(p_mr), format_p(p_benign), format_p(p_pets), format_p(p_miss)]
    for ax, ptext in zip(axes, p_texts):
        ax.text(0.5, 0.95, ptext, transform=ax.transAxes,
                ha="center", va="top", fontsize=9)
        
    
    plt.subplots_adjust(wspace=0.4)
    plt.tight_layout(pad=2.0)

    plt.show()

    # --- Text summary ---
    print(f"Prevalence: {prevalence:.2f}")
    print(f"Benign biopsies (per 1000): PET-first {benign_pet[0]:.1f} vs Biomarker-first {benign_bio[0]:.1f}  ({format_p(p_benign)})")
    print(f"Cancers in surveillance: PET-first {miss_pet[0]:.1f} vs Biomarker-first {miss_bio[0]:.1f}  ({format_p(p_miss)})")
    print(f"Malignancy among biopsies (%): PET-first {mr_pet[0]:.1f} vs Biomarker-first {mr_bio[0]:.1f}  ({format_p(p_mr)})")
    print(f"Total PET scans: PET-first {total_pet_pet[0]:.1f} vs Biomarker-first {total_pet_bio[0]:.1f}  ({format_p(p_pets)})")
        





out = widgets.Output()

def update(_=None):
    with out:
        out.clear_output(wait=True)
        run_simulation(
            prevalence=prevalence_slider.value,
            bio_sens=bio_sens_slider.value,
            bio_spec=bio_spec_slider.value,
            pet_sens=pet_sens_slider.value,
            pet_spec=pet_spec_slider.value,
        )

# Attach observers ONCE
for w in [prevalence_slider, bio_sens_slider, bio_spec_slider,
          pet_sens_slider, pet_spec_slider,
          biomarker_preset, pet_preset]:
    w.observe(update, names="value")

# Build your UI layout (what you already have)
ui = widgets.VBox([
    widgets.HTML("<b>Prevalence of Cancer</b>"),
    HBox([prevalence_slider]),
    widgets.HTML("<b>Biomarker Performance</b>"),
    HBox([biomarker_preset]),
    HBox([bio_sens_slider, bio_spec_slider]),
    widgets.HTML("<b>PET Performance</b>"),
    HBox([pet_preset]),
    HBox([pet_sens_slider, pet_spec_slider]),
    ], layout=Layout(
        border="2px solid #888",  # the box
        padding="12px",
        margin="8px",
        border_radius="8px",
        width="fit-content"
    )
)

display(ui, out)

# Run once at startup
update()


Output()